In [ ]:
import fastbook
fastbook.setup_book()

In [ ]:
#hide
from fastbook import *
from fastai.vision.widgets import *

In [ ]:
search_images_ddg

In [ ]:
face_problem_types = ['DARK CIRCLES ON FACE','DARK SPOTS ON FACE','EYE BAGS','DRY SKIN','OILY SKIN','FACE REDNESS',
          'DULL SKIN','ROUGH TEXTURE ON FACE','LARGE PORES ON FACE','UNDER-EYE WRINKLES','HORMONAL ACNE',
          'BLACKHEADS','SEBACEOUS FILAMENTS','FOREHEAD WRINKLES','RAZOR BUMPS']
path = Path('Skin_Problems')

In [ ]:
if not path.exists():
    path.mkdir()
    for face_problem_type in face_problem_types:
        dest = (path/face_problem_type)
        dest.mkdir(exist_ok=True)
        results = search_images_ddg(face_problem_type,max_images=500)
        download_images(dest, urls=results)

In [ ]:
fns = get_image_files(path)
fns

In [ ]:
failed = verify_images(fns)
failed

In [ ]:
failed.map(Path.unlink);

In [ ]:
face_problems = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.1, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128))

In [ ]:
dls = face_problems.dataloaders(path)

In [ ]:
face_problems = face_problems.new(item_tfms=Resize(128), batch_tfms=aug_transforms(mult=2))
dls = face_problems.dataloaders(path)
dls.train.show_batch(max_n=8, nrows=2, unique=True)

In [ ]:
face_problems = face_problems.new(
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms(),
)
dls = face_problems.dataloaders(path,bs=16)

In [ ]:
learn = cnn_learner(dls, resnet50, metrics=error_rate)
learn.fine_tune(5)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(1, nrows=1)

In [ ]:
learn.export()

In [ ]:
path = Path()
path.ls(file_exts='.pkl')

In [ ]:
learn_inf = load_learner(path/'export.pkl')

In [ ]:
learn_inf.predict('harmonal_acne.jpg')

In [ ]:
learn_inf.dls.vocab

In [ ]:
btn_upload = widgets.FileUpload()
btn_upload

In [ ]:
#hide
# For the book, we can't actually click an upload button, so we fake it
btn_upload = SimpleNamespace(data = ['harmonal_acne.jpg'])

In [ ]:
img = PILImage.create(btn_upload.data[-1])

In [ ]:
out_pl = widgets.Output()
out_pl.clear_output()
with out_pl: display(img.to_thumb(128,128))
out_pl

In [ ]:
pred,pred_idx,probs = learn_inf.predict(img)

In [ ]:
lbl_pred = widgets.Label()
lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'
lbl_pred

In [ ]:
btn_run = widgets.Button(description='Classify')
btn_run

In [ ]:
def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = learn_inf.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

btn_run.on_click(on_click_classify)

In [ ]:
#hide
#Putting back btn_upload to a widget for next cell
btn_upload = widgets.FileUpload()

In [ ]:
VBox([widgets.Label('Select your bear!'), 
      btn_upload, btn_run, out_pl, lbl_pred])

In [ ]:
labels = learn.dls.vocab
def predict(img):
    img = PILImage.create(img)
    pred,pred_idx,probs = learn.predict(img)
    return {labels[i]: float(probs[i]) for i in range(len(labels))}

In [ ]:
import gradio as gr
gr.Interface(fn=predict, inputs=gr.inputs.Image(shape=(512, 512)), outputs=gr.outputs.Label(num_top_classes=3)).launch(share=True)

In [ ]:
import gradio as gr
from fastai.vision.all import *
import skimage

learn = load_learner('export.pkl')

labels = learn.dls.vocab
def predict(img):
    img = PILImage.create(img)
    pred,pred_idx,probs = learn.predict(img)
    return {labels[i]: float(probs[i]) for i in range(len(labels))}

title = "Face condition Analyzer"
description = "A face condition detector trained on the custom dataset with fastai. Created using Gradio and HuggingFace Spaces."
examples = ['harmonal_acne.jpg','forehead_wrinkles.jpg','oily_skin.jpg']
enable_queue=True

gr.Interface(fn=predict,inputs=gr.inputs.Image(shape=(512, 512)),outputs=gr.outputs.Label(num_top_classes=3),title=title,
             description=description,examples=examples,enable_queue=enable_queue).launch(share=True)

In [ ]:
import gradio as gr
from fastai.vision.all import *
import skimage
import pathlib
import pandas as pd

plt = platform.system()
if plt == 'Linux': pathlib.WindowsPath = pathlib.PosixPath
title = "Face condition Analyzer"
description = "A face condition detector trained on the custom dataset with fastai. Created using Gradio and HuggingFace Spaces."
examples = [['harmonal_acne.jpg'],['forehead_wrinkles.jpg'],['oily_skin.jpg']]
enable_queue=True

with gr.Blocks(title=title,description=description,examples=examples,enable_queue=enable_queue) as demo:
    learn = load_learner('export.pkl')
    labels = learn.dls.vocab
    def predict(img):
        img = PILImage.create(img)
        pred,pred_idx,probs = learn.predict(img)
        return {labels[i]: float(probs[i]) for i in range(len(labels))}
    gr.Markdown("# Face Skin Analyzer")
    gr.Markdown("A face condition detector trained on the custom dataset with fastai. Created using Gradio and HuggingFace Spaces. Kindly upload a photo of your face.")
    with gr.Row():
        inputs = gr.inputs.Image(shape=(512, 512))
        outputs = gr.outputs.Label(num_top_classes=3)
    btn = gr.Button("Predict")
    btn.click(fn=predict, inputs=inputs, outputs=outputs)
    
    df=pd.read_excel("recommendation.xlsx")
    classes = df['class'].unique()
    with gr.Accordion("Find your skin condition using above analyzer and see the Recommended solutions",open=True):
        for c in classes:
            with gr.Accordion(c,open=False):
                df_temp = df[df['class']==c]
                with gr.Row():
                    for i,current_row in df_temp.iterrows():
                        with gr.Column():
                            html_box = gr.HTML("<a href='{}'><img src ='{}'></a>".format(current_row['profit_link'],current_row['product_image']))                   
demo.launch()